In [32]:
import GPUtil

gpus = GPUtil.getGPUs()
for gpu in gpus:
    print(f"ID: {gpu.id}")
    print(f"Name: {gpu.name}")
    print(f"Total Memory: {gpu.memoryTotal}MB")
    print(f"Free Memory: {gpu.memoryFree}MB")
    print(f"Used Memory: {gpu.memoryUsed}MB")
    print(f"Temperature: {gpu.temperature}C")
    print(f"UUID: {gpu.uuid}")
    print("-" * 20)

ID: 0
Name: NVIDIA RTX A4000
Total Memory: 16376.0MB
Free Memory: 16099.0MB
Used Memory: 8.0MB
Temperature: 34.0C
UUID: GPU-d82a5135-4b75-b424-83dd-f4a8abf14b78
--------------------
ID: 1
Name: NVIDIA RTX A4000
Total Memory: 16376.0MB
Free Memory: 16028.0MB
Used Memory: 77.0MB
Temperature: 38.0C
UUID: GPU-577e9411-9c64-6950-0fd4-c6d639a3c527
--------------------
ID: 2
Name: NVIDIA RTX A4000
Total Memory: 16376.0MB
Free Memory: 16099.0MB
Used Memory: 8.0MB
Temperature: 36.0C
UUID: GPU-7d6e9193-a87d-2471-5770-32ff710595b2
--------------------
ID: 3
Name: NVIDIA RTX A4000
Total Memory: 16376.0MB
Free Memory: 16099.0MB
Used Memory: 8.0MB
Temperature: 36.0C
UUID: GPU-c7492d31-eb01-59eb-3130-4796db256b3f
--------------------


In [33]:
continuous_series_names = [
                           'uncorrelated_gaussian_centered_sigma_0.1_KDE',
                           'uncorrelated_gaussian_centered_sigma_0.3_KDE',                           
                           'uncorrelated_gaussian_centered_sigma_0.5_KDE',
                           'uncorrelated_gaussian_centered_sigma_0.8_KDE',
                           'uncorrelated_gaussian_centered_sigma_0.1_histogram',
                           'uncorrelated_gaussian_centered_sigma_0.3_histogram',
                           'uncorrelated_gaussian_centered_sigma_0.5_histogram',
                           'uncorrelated_gaussian_centered_sigma_0.8_histogram',
                           'uncorrelated_random_PDF_l_0.1_KDE',
                           'uncorrelated_random_PDF_l_0.1_histogram',
                           'uncorrelated_random_PDF_l_0.4_histogram',
                           'uncorrelated_random_PDF_l_0.4_KDE',
                           'uncorrelated_random_PDF_l_0.02_KDE',
                           'uncorrelated_random_PDF_l_0.05_KDE',
                           'uncorrelated_random_PDF_l_0.1_KDE',
                           'uncorrelated_random_PDF_l_0.2_KDE',
                           'uncorrelated_random_PDF_l_0.3_KDE',
                           'uncorrelated_random_PDF_l_0.5_KDE',
                           'uncorrelated_random_PDF_l_0.02_histogram',
                           'uncorrelated_random_PDF_l_0.05_histogram',
                           'uncorrelated_random_PDF_l_0.1_histogram',
                           'uncorrelated_random_PDF_l_0.2_histogram',
                           'uncorrelated_random_PDF_l_0.3_histogram',
                           'uncorrelated_random_PDF_l_0.5_histogram',
                           ]
markov_chain_names = ['markov_chain']

import numpy as np

### Set up directory
import sys
import os
from pathlib import Path
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

from tqdm import tqdm
import pickle
import torch
import matplotlib.pyplot as plt

# Check if directory exists, if not create it
save_path = Path(parent_dir) / 'processed_series'
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
# Define the directory where the generated series are stored
generated_series_dir = Path(parent_dir) / 'generated_series'

In [34]:
# Initialize dictionaries to store the data for continuous series and Markov chains
continuous_series_task = {}
markov_chain_task = {}

# Loop through each file in the directory
for file in generated_series_dir.iterdir():
    # Check if a series is already processed
    if not (save_path / file.name).exists():
        # Extract the series name from the file name
        series_name = file.stem.rsplit('_', 1)[0]
        # If the series is a continuous series, load the data into the continuous_series_data dictionary
        if series_name in continuous_series_names:
            continuous_series_task[file.name] = pickle.load(file.open('rb'))
        # If the series is a Markov chain, load the data into the markov_chain_data dictionary
        elif series_name in markov_chain_names:
            markov_chain_task[file.name] = pickle.load(file.open('rb'))
        # If the series name is not recognized, raise an exception
        # else:
        #     raise Exception(f"Unrecognized series name: {series_name}")
        


In [35]:
print(continuous_series_task.keys())
print(markov_chain_task.keys())

dict_keys(['uncorrelated_gaussian_centered_sigma_0.1_KDE_17.pkl'])
dict_keys([])


### Analyze Multi Digit series

In [36]:
import importlib
import baseline_models
importlib.reload(baseline_models)

from baseline_models import *


In [37]:
for series_name, series_dict in sorted(continuous_series_task.items()):   
    llama_size = series_dict['llama_size']
    prec = series_dict['prec']
    kernel = series_dict['kernel']
    rescaled_full_series = series_dict['rescaled_full_series']
    bw_list = series_dict.get('bw_list', None)
    if llama_size == 'KDE':
        print("Processing ", series_name)
        PDF_list = KDE_for_series(rescaled_full_series,kernel,prec,bw_list)
        series_dict['PDF_list'] = PDF_list
        save_name = os.path.join(save_path, series_name)
        with open(save_name, 'wb') as f:
            pickle.dump(series_dict, f)
            
    if llama_size == 'histogram':
        print("Processing ", series_name)
        PDF_list = histogram_for_series(rescaled_full_series,prec)
        series_dict['PDF_list'] = PDF_list
        save_name = os.path.join(save_path, series_name)
        with open(save_name, 'wb') as f:
            pickle.dump(series_dict, f)            


Processing  uncorrelated_gaussian_centered_sigma_0.1_KDE_17.pkl
